## R-alpha

In [ ]:
!pip install smartapi-python
!pip install websocket-client
!pip install logzero
!pip install pyotp
!pip install requests
!pip install numpy
!pip install pandas
!pip install yfinance

In [ ]:
# package import statement
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger
from connect_api import connect_api, create_api_session
from data import get_data_stream, token_lookup, get_instrument_list, hist_data, hist_data_from_now
from strategy import expected_buy_price, expected_sell_price, trend_detection, exit_long, exit_short
from order import place_limit_order, place_market_order
import json
import threading
import time   

In [ ]:
api_key = '2VTNvx7z'
username = 'R258447'
pwd = '1499'
token_id = '3URP5C2ZUQSYTSEFELMM2CFJGM'

In [ ]:
smartApi = connect_api(api_key)
data = create_api_session(token_id, username, pwd, smartApi)

### Get Ticker Data

In [ ]:
instrument_list = get_instrument_list()

ticker_list = ["ITC"]
token_list_0 = []

for ticker in ticker_list:
    token = token_lookup(ticker, instrument_list, exchange="NSE")
    token_list_0.append(token)

token_list = [
        {
            "exchangeType": 1,
            "tokens": token_list_0
        }
    ]


print(token_list)

## Create Live Data Stream

In [ ]:
correlation_id = "stream_2" #any string value which will help identify the specific streaming in case of concurrent streaming
action = 1 #1 subscribe, 0 unsubscribe
mode = 1 #1 for LTP, 2 for Quote and 3 for SnapQuote
    
sws = get_data_stream(token_id, api_key, username, smartApi.getfeedToken(), token_list, correlation_id, action, mode)

## Implement Strategy

## Momentum Strategy

In [ ]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta

# Alpha Vantage API parameters
API_KEY = '5PTSVQOUZRRRZN7S'
symbol = 'ITC'
interval = '5min'  # 5-minute interval data
outputsize = 'full'  # Full intraday data

# Initialize variables
pos1 = int(0)  # 1 for long position, -1 for short position, 0 for no position
capital = 1000  # Initial capital in USD
current_position_value = 0
last_price = 0

# Define parameters
lookback_period = 12  # Number of intervals to look back for momentum calculation
buy_threshold = 0.0000003  # Buy if the price increases by more than 0.5% in the lookback period
sell_threshold = -0.005  # Sell if the price decreases by more than 0.5% in the lookback period

df = pd.DataFrame(columns=['Price','Returns'])

# Fetch intraday price data from Alpha Vantage

def momentum_strategy(tick):
    token = tick['token']
    ltp = tick['last_traded_price']/100
    df.loc[len(df),'Price'] = ltp
    print(ltp)
    # Calculate momentum
    df['Returns'] = df['Price'].pct_change()
    df['Momentum'] = df['Returns'].rolling(window=lookback_period).sum()

    # Implement momentum strategy
    
    price = df['Price'].iloc[-1]
    momentum = df['Momentum'].iloc[-1]
  
    
    if len(df) > lookback_period:
        if momentum > buy_threshold:
            print('hiiii')
            # if pos1 != 1:
            current_position_value = capital
            # pos1 = 1
            last_price = price
            print(f"Buying at {price}")

        elif momentum < sell_threshold:
            # if pos1 != -1:  # Enter short position
            current_position_value = -capital
            # pos1 = -1
            last_price = price
            print(f"Selling short at {price}")



def on_data(ws, message):
    momentum_strategy(message)

sws.on_data = on_data

## misc

In [ ]:
correlation_id = "stream_1"
action = 1
mode = 1
token_list = [
        {
            "exchangeType": 1,
            "tokens": ["26009"]
        }
    ]

sws = get_data_stream(token_id, api_key, username, smartApi.getfeedToken(), token_list, correlation_id, action, mode)

In [ ]:
# Create a new thread and pass the function to it
my_thread = threading.Thread(target=sws.connect())

# Start the thread
my_thread.start()
time.sleep(10)

In [ ]:
place_market_order(instrument_list,ticker="ITC",buy_sell="BUY",price=0,quantity=1,obj=smartApi, sl=0,sqof=0,exchange="NSE")

In [ ]:
nifty_50 = hist_data_from_now(smartApi, "NIFTYBEES", 2000, "ONE_DAY", instrument_list)

In [ ]:
nifty_50.to_csv("niftybees_2000day_1day.csv")

In [ ]:
instrument_list[0]